### Define Data

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("clustered_df.csv")

In [28]:
selected_cluster = 3

df[f'class{selected_cluster}_binary'] = (df['class'] == selected_cluster) *1
X = df.drop(labels = ['Unnamed: 0.1', 'Unnamed: 0', 'track_id', 'artists', 'album_name',
       'track_name','track_genre', 'class'], axis=1)
y = X.pop(f'class{selected_cluster}_binary')

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size = 0.2)

### Assess Models

In [45]:
#Negative Model
y_test.value_counts()[0]/len(y_test)

0.8245175438596491

In [44]:
#Logistic model
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(random_state=42)
log_clf.fit(X_train, y_train)
yhat = log_clf.predict(X_test)
(yhat == y_test).mean()

0.8244736842105264

In [31]:
#Adaboost model
from sklearn.ensemble import AdaBoostClassifier

abc_clf = AdaBoostClassifier(n_estimators=100, algorithm="SAMME", random_state=42)
abc_clf.fit(X_train, y_train)
yhat = abc_clf.predict(X_test)
(y_test == yhat).mean()

0.9732017543859649

In [43]:
#SVC model
from sklearn import svm

svc_clf = svm.SVC()
svc_clf.fit(X_train, y_train)
yhat = svc_clf.predict(X_test)
(y_test == yhat).mean()

0.8245175438596491